# Graphiti + LangGraph + Lapa LLM Demo

This notebook demonstrates an AI agent with long-term memory using:
- **Lapa LLM** - Ukrainian language model via hosted Lapathon API
- **Graphiti** - Temporal knowledge graph for memory
- **LangGraph** - Agent orchestration
- **Neo4j** - Graph database storage

## 1. Setup and Imports

In [1]:
import asyncio
import logging
from datetime import datetime
from langchain_core.messages import HumanMessage

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Import our modules
from config.settings import settings
from clients.llm_client import get_llm_client
from clients.graphiti_client import get_graphiti_client
from agent.graph import get_agent_app
from agent.state import create_initial_state
from utils.langsmith_setup import setup_langsmith

# Ініціалізація LangSmith
setup_langsmith()
print("✅ Imports successful")

✅ LangSmith tracing enabled for project: pr-potable-measles-91
✅ Imports successful


## 2. Check Neo4j Status

Verify that Neo4j is running before starting the demo

In [2]:
async def check_neo4j():
    """Verify Neo4j connection"""
    try:
        from neo4j import AsyncGraphDatabase
        driver = AsyncGraphDatabase.driver(
            settings.neo4j_uri,
            auth=(settings.neo4j_user, settings.neo4j_password)
        )
        async with driver.session() as session:
            await session.run("RETURN 1")
        await driver.close()
        print("✅ Neo4j is running")
        return True
    except Exception as e:
        print(f"❌ Neo4j not accessible: {e}")
        print("   Start with: docker-compose up -d")
        return False

await check_neo4j()

✅ Neo4j is running


True

## 3. Initialize Clients

In [3]:
# Initialize LLM client
llm_client = get_llm_client()
print(f"✅ LLM Client initialized: {llm_client.model_name}")

# Initialize Graphiti client
graphiti_client = await get_graphiti_client()
# await graphiti_client.initialize()
print("✅ Graphiti Client initialized", graphiti_client._initialized)

# Get agent app
agent = get_agent_app()
print("✅ Agent Graph compiled")

INFO:clients.llm_client:Using vLLM at http://146.59.127.106:4000 with model lapa
INFO:clients.graphiti_client:Initializing Graphiti client...
INFO:clients.graphiti_client:Using hosted Qwen embeddings
INFO:clients.hosted_embedder:Initializing hosted embedder: text-embedding-qwen
INFO:clients.hosted_embedder:API URL: http://146.59.127.106:4000


✅ LLM Client initialized: lapa


INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps
ERROR:graphiti_core.driver.neo4j_driver:Error executing Neo4j query: {neo4j_code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=5, name='community_uuid', type='RANGE', schema=(:Community {uuid}), indexProvider='range-1.0' )'.} {gql_status: 50N42} {gql_status_description: error: general processing exception - unexpected error. Unexpected error has occurred. See debug log for details.}
CREATE INDEX community_uuid IF NOT EXISTS FOR (n:Community) ON (n.uuid)
{'database_': 'neo4j'}
ERROR:graphiti_core.driver.neo4j_driver:Error executing Neo4j query: {neo4j_code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=6, name='has_member_uuid', type='RANGE', schema=()-[:HAS_MEMBER {uuid}]-(), indexProvider='range-1.0' )'.} {gql_status: 50N42} {gql_status_description: error: general process

✅ Graphiti Client initialized True
✅ Agent Graph compiled


INFO:neo4j.notifications:Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful completion - index or constraint already exists. The command 'CREATE FULLTEXT INDEX community_name IF NOT EXISTS FOR (e:Community) ON EACH [e.name, e.group_id]' has no effect. The index or constraint specified by 'FULLTEXT INDEX community_name FOR (e:Community) ON EACH [e.name, e.group_id]' already exists.", position=None, raw_classification='SCHEMA', classification=<NotificationClassification.SCHEMA: 'SCHEMA'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'SCHEMA', '_severity': 'INFORMATION', 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'CREATE FULLTEXT INDEX community_name IF NOT EXISTS\n        FOR (n:Community) ON EACH [n.name, n.group_id]'
INFO:neo4j.notifications:Received notification from DBMS server: <GqlStatusObject gql_status='00N

## 4. Test LLM Connection

Let's verify that our LLM is working and responds in Ukrainian

In [4]:
test_messages = [
    {"role": "system", "content": "Ти - корисний AI асистент."},
    {"role": "user", "content": "Привіт! Як справи?"}
]

response = await llm_client.generate_async(test_messages)
print("LLM Response:")
print(response)

INFO:httpx:HTTP Request: POST http://146.59.127.106:4000/chat/completions "HTTP/1.1 200 OK"
INFO:clients.llm_client:Token usage: {'prompt_tokens': 21, 'completion_tokens': 29, 'total_tokens': 50}


LLM Response:
Привіт! Я радий/рада тебе бачити. Я тут, щоб допомогти тобі з будь-якими питаннями. Як я можу тобі допомогти сьогодні?


## 5. First Conversation: Building Memory

In this conversation, we'll introduce ourselves and provide some personal information

In [5]:
# Create user configuration
USER_ID = "test_user_1"
SESSION_ID = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# First message: introduce yourself
first_message = HumanMessage(content="Привіт! Мене звати Олександр, я з Києва і працюю програмістом. Мені 32 роки. В мене є стартап SMAQ.")

# Create initial state
config = {"configurable": {"thread_id": SESSION_ID}}

# Run agent
result = await agent.ainvoke(
    {
        "messages": [first_message],
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "retrieved_context": None,
        "timestamp": datetime.now(),
        "current_query": None,
        "needs_memory_update": False,
        "search_results": None,
        "message_count": 0
    },
    config=config
)

print("\n" + "="*60)
print("User: Привіт! Мене звати Олександр, я з Києва і працюю програмістом.")
print("="*60)
print(f"Agent: {result['messages'][-1].content}")
print("="*60 + "\n")

INFO:agent.nodes:=== Retrieve Memory Node ===
INFO:agent.nodes:User query: Привіт! Мене звати Олександр, я з Києва і працюю програмістом. Мені 32 роки. В мене є стартап SMAQ....
INFO:httpx:HTTP Request: POST http://146.59.127.106:4000/embeddings "HTTP/1.1 200 OK"
INFO:clients.hosted_embedder:Embedding dimension: 4096
INFO:clients.graphiti_client:Search returned 0 results for query: Привіт! Мене звати Олександр, я з Києва і працюю програмістом. Мені 32 роки. В мене є стартап SMAQ.
INFO:agent.nodes:No relevant memories found
INFO:agent.nodes:=== Generate Response Node ===
INFO:agent.nodes:Generating response with 2 messages in context
INFO:httpx:HTTP Request: POST http://146.59.127.106:4000/chat/completions "HTTP/1.1 200 OK"
INFO:clients.llm_client:Token usage: {'prompt_tokens': 90, 'completion_tokens': 55, 'total_tokens': 145}
INFO:agent.nodes:Generated response: Привіт, Олександре! Дуже приємно з тобою познайомитися. Я радий/рада допомогти тобі з будь-якими пит...
INFO:agent.nodes:=== 


User: Привіт! Мене звати Олександр, я з Києва і працюю програмістом.
Agent: Привіт, Олександре! Дуже приємно з тобою познайомитися. Я радий/рада допомогти тобі з будь-якими питаннями. Якщо в тебе є якісь конкретні теми, які тебе цікавлять, або якщо тобі потрібна допомога з чимось, не соромся питати. Я тут, щоб допомогти!



In [6]:
# Get graph statistics
stats = await graphiti_client.get_graph_stats()
print(f"📊 Graph Memory Stats:")
print(f"   Nodes: {stats['node_count']}")
print(f"   Relationships: {stats['relationship_count']}")
print(f"\n💡 The agent is learning and building a knowledge graph!")

📊 Graph Memory Stats:
   Nodes: 3
   Relationships: 8

💡 The agent is learning and building a knowledge graph!


In [7]:
# Get graph statistics
stats = await graphiti_client.get_graph_stats()
print(f"📊 Graph Stats:")
print(f"   Nodes: {stats['node_count']}")
print(f"   Relationships: {stats['relationship_count']}")

# Search for specific information
search_results = await graphiti_client.search("Олександр Київ")
print(f"\n🔍 Search results for 'Олександр Київ': {len(search_results)} found")
for i, search_item in enumerate(search_results[:3], 1):
    print(f"   {i}. {search_item.get('content', 'N/A')[:100]}...")

📊 Graph Stats:
   Nodes: 3
   Relationships: 8


INFO:httpx:HTTP Request: POST http://146.59.127.106:4000/embeddings "HTTP/1.1 200 OK"
INFO:clients.graphiti_client:Search returned 6 results for query: Олександр Київ



🔍 Search results for 'Олександр Київ': 6 found
   1. uuid='8b66c389-ad31-4c02-9653-efc0f41e115f' group_id='' source_node_uuid='04a42342-aa8e-4428-b177-80...
   2. uuid='20356d51-2ea1-49d9-8f09-133e116e83ab' group_id='' source_node_uuid='04a42342-aa8e-4428-b177-80...
   3. uuid='c4cc0389-8e67-4e3c-a407-e403556006f0' group_id='' source_node_uuid='04a42342-aa8e-4428-b177-80...


## 7. Second Conversation: Testing Memory Recall

Now let's ask a question that requires recalling information from previous conversation

## 8. Third Conversation: More Complex Query

## 9. Visualize Knowledge Graph

Let's query Neo4j directly to see what entities and relationships were created

## Summary

### What We Demonstrated:
1. ✅ **Hosted Lapa LLM** - Ukrainian language model via Lapathon API
2. ✅ **Hosted Qwen Embeddings** - Semantic search using hosted embeddings
3. ✅ **Graphiti Memory** - Temporal knowledge graph for long-term memory
4. ✅ **LangGraph Agent** - Three-node pipeline (retrieve → generate → save)
5. ✅ **Memory Recall** - Context-aware responses using graph memory

### Architecture:
- **LLM**: Lapa model @ http://146.59.127.106:4000
- **Embeddings**: text-embedding-qwen (hosted)
- **Memory**: Graphiti + Neo4j graph database
- **Agent**: LangGraph with persistent state

### Next Steps:
1. Explore Neo4j Browser: http://localhost:7474
2. Try different conversation topics
3. Test memory across multiple sessions
4. Experiment with Mamay model (change VLLM_MODEL_NAME=mamay in .env)

## 10. Summary and Next Steps

### What We Demonstrated:
1. ✅ Lapa LLM integration via vLLM with structured outputs
2. ✅ Graphiti knowledge graph for long-term memory
3. ✅ LangGraph agent orchestration with state management
4. ✅ Memory retrieval and contextual responses
5. ✅ Graph visualization and querying

### Key Features:
- **Temporal Memory**: Graphiti tracks when information was learned
- **Semantic Search**: Hybrid search (embeddings + BM25 + graph traversal)
- **Context Awareness**: Agent uses retrieved memories to personalize responses
- **Ukrainian Support**: Lapa LLM optimized for Ukrainian language

### Next Steps:
1. Add more conversations to build richer memory
2. Experiment with different query types
3. Visualize graph in Neo4j Browser (http://localhost:7474)
4. Test with multiple users/sessions
5. Implement memory cleanup strategies for old data

## 11. Cleanup (Optional)

In [ ]:
# Uncomment to clear all graph data
# from neo4j import AsyncGraphDatabase
#
# async def clear_graph():
#     driver = AsyncGraphDatabase.driver(
#         settings.neo4j_uri,
#         auth=(settings.neo4j_user, settings.neo4j_password)
#     )
#     async with driver.session(database=settings.neo4j_database) as session:
#         await session.run("MATCH (n) DETACH DELETE n")
#     await driver.close()
#     print("✅ Graph cleared")
#
# await clear_graph()